In [4]:
# code is adapted from CADDA and braincode
from audioop import reverse
from enum import auto
from numbers import Real
from operator import invert
import random
import numpy as np
import torch
import torch.nn as nn
from scipy.interpolate import Rbf
from scipy.spatial.transform import Rotation
from sklearn.utils import check_random_state
from torch.fft import fft, ifft
from torch.nn.functional import dropout2d, pad, one_hot
from torch.distributions import Normal
from mne.filter import notch_filter
from mne.channels.interpolation import _make_interpolation_matrix
from mne.channels import make_standard_montage
import matplotlib.pyplot as plt
from ecgdetectors import Detectors
from scipy.interpolate import CubicSpline

target_dir = 'subdiag'
num_class = 23
n_fold = 10
predict_mode = 'test'

In [22]:
### torch adaptive test
bs,ch,seqlen = 1,1,100
signal = torch.randn(bs,seqlen,ch)
#print(signal)
windowed_sig = torch.nn.functional.avg_pool1d(signal.view(1,1,seqlen),kernel_size=10, stride=1, padding=5,count_include_pad=False).view(1,-1)[:,:seqlen]
print(windowed_sig.shape,windowed_sig)
quant_score = torch.quantile(windowed_sig[0],0.8)
print('quant: ',quant_score)
#keep adapt ways
select_windows = (windowed_sig[0] >= quant_score).nonzero(as_tuple=True)[0].detach().cpu().numpy()
#select_windows = select_windows + 10 // 2
#keep auto ways
passed_windows = []
for x in range(100):
    x1 = np.clip(x - 10 // 2, 0, seqlen)
    x2 = np.clip(x + 10 // 2, 0, seqlen)
    reg_mean = signal[0,x1: x2].mean()
    if reg_mean >= quant_score:
        passed_windows.append(reg_mean)
passed_windows = np.array(passed_windows)
print('Compare two result')
print('Adapt ',windowed_sig[0,select_windows])
print('Auto ',passed_windows)

torch.Size([1, 100]) tensor([[ 0.0633, -0.1828, -0.1010,  0.1154,  0.0248,  0.2018, -0.0248,  0.1244,
          0.2477,  0.1866,  0.1078,  0.2721,  0.3446,  0.3025,  0.3193,  0.0182,
          0.4350,  0.2556,  0.2274,  0.0851,  0.1164,  0.1907,  0.1009,  0.0985,
          0.2254,  0.4081,  0.0429,  0.0757, -0.0316,  0.2798,  0.4592,  0.4829,
          0.4250,  0.3467,  0.3093,  0.0192,  0.0951,  0.0831,  0.0982, -0.0779,
         -0.1436, -0.3270, -0.1693, -0.3200, -0.3753, -0.2109, -0.3879, -0.3733,
         -0.5053, -0.6190, -0.6050, -0.5224, -0.6313, -0.5602, -0.4642, -0.4588,
         -0.3159, -0.4049, -0.0823, -0.0616,  0.0677,  0.1508,  0.1642,  0.2318,
          0.0893,  0.1577,  0.0687,  0.1511, -0.0586, -0.0329, -0.0433, -0.0052,
         -0.1156, -0.1102,  0.0600,  0.1076,  0.1307,  0.2685,  0.1922,  0.2164,
          0.0936, -0.1973, -0.1023, -0.1345, -0.2304, -0.3110, -0.1868, -0.1859,
         -0.0738, -0.0461, -0.0772,  0.0519,  0.0598, -0.0221,  0.0557,  0.1445,
       

In [ ]:
dummy = np.array([0,1,2,3,3,6,4,6,6,6,6,6,6,2,2,1,3,5,5,5])
